In [ ]:
import os
import numpy as np
from src.jobs import Train

repo_dir = '/media/cam/CAPSTONEDB/Enceladus/'
os.chdir(repo_dir)

args = {
    'seed'                : np.random.seed(69),
    'tfrecords_dir'       : '/home/cam/Documents/database_tools/data-2022-09-18/records/',
    'data_method'         : 'PLETH, ABP Values',
    'out_dir'             : 'output/',
    'model_dir'           : 'model/',
    'epochs'              : 10,
    'batch_size'          : 32,
    'val_steps'           : None,
    'steps_per_epoch'     : None,
    'es_patience'         : 3,
    'test_steps'          : None,
    'use_wandb_tracking'  : False,
    'wandb_entity'        : None,
    'wandb_project'       : None,
    'use_multiprocessing' : False, 
}

Train(args=args).run()

In [ ]:
import os
import keras
import tensorflow as tf
from database_tools.utils import ReadTFRecords

repo_dir = '/home/cam/Documents/DeepBP/'
os.chdir(repo_dir)

model = keras.models.load_model('output/model/')

In [ ]:
data_splits = ReadTFRecords(data_dir=f'../database_tools/data-2022-09-18/records/',
                            method='PLETH, ABP Values',
                            n_cores=10,
                            AUTOTUNE=tf.data.experimental.AUTOTUNE).run()
train = data_splits['train']
val = data_splits['val']
test = data_splits['test']

In [ ]:
preds = []
y = []
for feature in test.take(5000):
    feature
    x = tf.expand_dims(feature[0], axis=0)
    y.append(feature[1])
    preds.append(model.predict(x))

In [ ]:
sbp_true = [float(label) for label, _ in y]
dbp_true = [float(label) for _, label in y]
sbp_pred = [float(label[0][0]) for label in preds]
dbp_pred = [float(label[0][1]) for label in preds]

In [ ]:
def denormalize(x):
    return (x * (250 - 25)) + 25

sbp_true = map(denormalize, sbp_true)
dbp_true = map(denormalize, dbp_true)
sbp_pred = map(denormalize, sbp_pred)
dbp_pred = map(denormalize, dbp_pred)

In [ ]:
import pandas as pd

df = pd.DataFrame({'true': sbp_true,
                   'pred': sbp_pred})
df.plot.scatter(x='pred', y='true')

In [ ]:
df = pd.DataFrame({'true': dbp_true,
                   'pred': dbp_pred})
df.plot.scatter(x='pred', y='true')

In [ ]:
from DeepBP.utils.models import create_model

config = {
    'model'         : 'deepbp_2',
    'input_shape'   : (625, 1),
    'lr'            : 0.00001,
    'decay'         : 0.00001,
    'kernel_1'      : (7),
    'filters_1'     : 64,
    'stride_1'      : (2),
    'max_pooling_1' : (3),
    'kernel_2'      : [(3), (3)],
    'filters_2'     : [64, 64],
    'kernel_3'      : [(3), (3)],
    'filters_3'     : [128, 128],
    'stride_3'      : (2),
    'kernel_4'      : [(3), (3)],
    'filters_4'     : [256, 256],
    'stride_4'      : (2),
    'kernel_5'      : [(3), (3)],
    'filters_5'     : [512, 512],
    'stride_5'      : (2),
    'pooling'       : (2),
    'dense_0'       : 1024,
    'frame_length'  : 125,
    'frame_step'    : 8,
    'st_units'      : 64,
    'cnn_st_units'  : 64,
    'dropout_1'     : 0.15,
    'dropout_2'     : 0.15,
    'dense_1'       : 32,
    'dense_2'       : 32,
}

model = create_model(config)
model.summary()

In [ ]:
# [(W - K + 2P) / S ] + 1

# input
x = 625

# kernel_1, strides_1
x = ((x - 10 + (2 * 0)) / 3) + 1
x


In [ ]:
from specutils.analysis import snr
from specutils import Spectrum1D

samples = np.arange(2 * 60)
x = np.sin(samples)
Spectrum1D(x)

In [ ]:
x = np.arange(20000)
len(x)

In [ ]:
fs = 125
l = fs * 60
i = 3
len(x[i*l:(i+1)*l])

In [ ]:
import pandas as pd

pd.options.plotting.backend = "plotly"

df = pd.DataFrame({'x': x, 'y': y })
df[['x', 'y']].plot()

In [ ]:
df = pd.read_json('../database_tools/data-2022-09-18/mimic3/mimic3_00000000.jsonlines', lines=True)
df.head(2)

In [ ]:
x = np.array(df.loc[0, 'pleth'])

In [ ]:
ppg_findpeaks(x, 125)

In [ ]:
pd.Series(df.loc[1, 'pleth']).plot()